# Example Full Pipeline
### This pipeline shows how to:
1. Connect to and stream from an EEG device (board)
2. Start the SSVEPStimulus
3. Segment the EEG data
4. Filter the EEG data
5. Classify the EEG data

### Importing Modules:

In [4]:
import sys
sys.path.append("..") # So that it can find the modules

# Assuming the other modules (brainflow_stream, filtering, segmentation, classification) are available
from modules.brainflow_stream import *
from modules.filtering import *
from modules.brainflow_filtering import *
from modules.segmentation import *
from modules.classification import *
from modules.ssvep_stim import *

### Settings & Variables

In [5]:
serial_port = 'COM4' 
board_id = BoardIds.SYNTHETIC_BOARD # Example Board - replace with actual board from: https://brainflow.readthedocs.io/en/stable/UserAPI.html#brainflow-board-shim
box_frequencies = [9.25, 11.25, 13.25, 15.25] # How many boxes and what frequencies
box_texts = ['Right', 'Left', 'Up', 'Down'] # What you want the boxes to say
box_text_indices = [0, 1, 3, 2] # Which boxes you want to display which text (clockwise from top)
display_index = 0 # Which monitor to display the stimulus on
display_mode = 'both' # Mode of display ('freq', 'text', 'both').
monitor_name = 'testMonitor' # Name of monitor (leaving this is fine -> may need to install standalone psychopy)
segment_duration = 2 # The number of seconds for each segment of data to be processed
harmonics = np.arange(1, 4) # Generates the 1st, 2nd, & 3rd Harmonics
highcut = 50 # Filter out data above 50 Hz
lowcut = 0.1 # Filter out data below 0.1 Hz


# Static Variables - Probably don't need to touch :)
sampling_rate = BoardShim.get_sampling_rate(board_id) # Find sampling rate of brainflow board
n_samples = sampling_rate * segment_duration # Calculates the number of samples in each segment
eeg_channels = BoardShim.get_eeg_names(board_id)
channel_names = ["O1", "O2", "Oz", "Pz", "P3", "P4", "POz", "P1"]
channel_mapping = dict(zip(eeg_channels, channel_names))

# Show board information
print(f"Channel Mapping: {channel_mapping}")
print(f"Sampling Rate: {sampling_rate}")
print(f"Segment Duration: {segment_duration}")

Channel Mapping: {'Fz': 'O1', 'C3': 'O2', 'Cz': 'Oz', 'C4': 'Pz', 'Pz': 'P3', 'PO7': 'P4', 'Oz': 'POz', 'PO8': 'P1'}
Sampling Rate: 250
Segment Duration: 2


## Pipeline

In [6]:
# Initialize the board
board = BrainFlowBoardSetup(board_id, serial_port)

# Start data stream
board.setup()

# Initialize the segmenter object
segmenter_obj = Segmentation(board, segment_duration=2)

# Initialize the filter object
filter_obj = Filtering(sampling_rate)

# Initialize the SSVEP Stimulus
stimulus_process = SSVEPStimulusRunner(box_frequencies=box_frequencies, 
                                        box_texts=box_texts, 
                                        box_text_indices=box_text_indices,
                                        display_index=display_index, 
                                        display_mode=display_mode, 
                                        monitor_name = monitor_name)

# Start the stimulus presentation process
stimulus_process.start()

# Retreive the calculated actual stimulus frequencies (necessary for CCA)
actual_frequencies = stimulus_process.get_actual_frequencies()

print(f"Actual Frequencies: {actual_frequencies}")

# Initialize the classifier (CCA)
cca_classifier = SSVEPClassifier(frequencies = actual_frequencies, 
                                    harmonics = harmonics, 
                                    sampling_rate = sampling_rate, 
                                    n_samples = n_samples, 
                                    method='CCA', 
                                    stack_harmonics=True)


# Start the processing loop!
start_time = time.time()

while time.time() - start_time < 30: # run for 30 seconds
    
    segment = segmenter_obj.get_segment_time()
    
    if segment is None:
        time.sleep(0.1)  # Wait a little before trying again
        continue

    eeg_segment = segment[:8, :]  # Assuming 8 channels, adjust if necessary
    
    filtered_segment = filter_obj.bandpass_filter(eeg_segment,
                                                  highcut=highcut,
                                                  lowcut=lowcut,
                                                  order=4  # Parameter adjusts rolloff of filter (higher = faster dropoff)
                                                  )
    
    detected_freq, correlation = cca_classifier(filtered_segment)
    
    print(f"Detected frequency using CCA: {detected_freq} Hz with correlation: {correlation:.3f}")
    
    ########
    # Insert output code here - i.e., if detected_freq = x: <perform action>
    #######

    time.sleep(0.1)

# Stop the SSVEPStimulus process
stimulus_process.stop()

# Stop data stream
board.stop()

Board setup and streaming started successfully
Actual Frequencies: [9.25, 11.28, 13.12, 15.24]
Detected frequency using CCA: 15.24 Hz with correlation: 0.524
Detected frequency using CCA: 15.24 Hz with correlation: 0.521
Detected frequency using CCA: 15.24 Hz with correlation: 0.517
Detected frequency using CCA: 15.24 Hz with correlation: 0.614
Detected frequency using CCA: 15.24 Hz with correlation: 0.643
Detected frequency using CCA: 15.24 Hz with correlation: 0.594
Detected frequency using CCA: 15.24 Hz with correlation: 0.601
Detected frequency using CCA: 15.24 Hz with correlation: 0.622
Detected frequency using CCA: 15.24 Hz with correlation: 0.574
Detected frequency using CCA: 15.24 Hz with correlation: 0.608
Detected frequency using CCA: 15.24 Hz with correlation: 0.565
Detected frequency using CCA: 15.24 Hz with correlation: 0.586
Detected frequency using CCA: 15.24 Hz with correlation: 0.589
Detected frequency using CCA: 15.24 Hz with correlation: 0.599
Detected frequency usin